In [ ]:
!pip install fer
!pip install ffmpeg moviepy
!pip install opencv-python
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=6e3e55a3da10f0d95c5133954016f04b4eb765da04cbac93f4ecc8ad6371d73b
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadconten

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import os
import re
from PIL import Image
# from fer import FER
import json
from deepface import DeepFace
import glob

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
def detect_faces(image_path, output_dir):
    # Load the YOLOv3 model
    net = cv2.dnn.readNet("/content/drive/MyDrive/Final Code/yolov3-wider_16000.weights", "/content/drive/MyDrive/Final Code/yolov3-face.cfg")

    # Define the classes
    classes = ["face"]

    # Read in an image
    image = cv2.imread(image_path)
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), (0, 0, 0), True, crop=False)

    # Run the model on the image
    net.setInput(blob)
    output_layers = net.getUnconnectedOutLayersNames()
    layer_outputs = net.forward(output_layers)

    # Extract the bounding boxes
    boxes = []
    confidences = []
    class_ids = []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.80 and class_id == 0:
                center_x = int(detection[0] * image.shape[1])
                center_y = int(detection[1] * image.shape[0])
                width = int(detection[2] * image.shape[1])
                height = int(detection[3] * image.shape[0])
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)

                # Check if current bounding box is inside any previous bounding boxes
                is_duplicate = False
                for prev_box in boxes:
                    if left >= prev_box[0] and top >= prev_box[1] and (left + width) <= (prev_box[0] + prev_box[2]) and (top + height) <= (prev_box[1] + prev_box[3]):
                        is_duplicate = True
                        break
                if not is_duplicate:
                    boxes.append([left, top, width, height])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

    # Draw the bounding boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.6)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            color = colors[i]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, classes[class_ids[i]], (x, y - 5), font, 1, color, 2)

            # Print the coordinates of the bounding box
            print(f"Face {i+1}: x={x}, y={y}, w={w}, h={h}")
            output_filename = os.path.join(output_dir, f"{image_name}_output_coordinates.txt")
            cv2.imwrite(os.path.join(output_dir, f"{image_name}_output.jpg"), image)
            with open(output_filename, "a") as f:
                f.write(f"Face {i+1}: x={x}, y={y}, w={w}, h={h}\n")



def extract_faces(image_path, coordinates_path, output_dir):
    img = Image.open(image_path)
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    # Create the output directory for the input image if it doesn't exist
    output_image_dir = os.path.join(output_dir, image_name)
    os.makedirs(output_image_dir, exist_ok=True)

    # Open the file with bounding box coordinates
    with open(coordinates_path, 'r') as f:
        boxes = f.readlines()

    # Convert the boxes list to a set of tuples to remove duplicates
    boxes = set(tuple(box.strip().split(',')) for box in boxes)

    # Define a regular expression pattern to match the bounding box coordinates
    pattern = r'x=(\d+), y=(\d+), w=(\d+), h=(\d+)'

    # Extract the boxed parts of the image and save them as separate images
    for i, box in enumerate(boxes):
        # Use regular expressions to extract the box coordinates
        match = re.search(pattern, ','.join(box))
        if not match:
            # Skip the current box if it doesn't match the pattern
            print(f"Box {box} in {coordinates_path} does not match the expected pattern")
            continue
        x, y, w, h = map(int, match.groups())

        # Extract the boxed part of the image
        box_img = img.crop((x, y, x+w, y+h))

        # Save the boxed part as a separate image in the output directory for the input image
        output_image_path = os.path.join(output_image_dir, f'{image_name}_face_{i}.jpg')
        box_img.save(output_image_path)

def analyze_emotion_deepface(img_path, input_dir, output_dir):
    # Perform emotion recognition
    result = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)

    # Get the maximum emotion and its score
    max_emotion = sorted(result[0]["emotion"].items(), key=lambda x: x[1], reverse=True)[0]
    max_emotion_name, max_emotion_score = max_emotion

    # Create the output image name
    image_name = os.path.splitext(os.path.basename(img_path))[0]
    output_image_name = f"{image_name}_{max_emotion_name}_{max_emotion_score:.2f}.jpg"

    # Get the folder name of the input image
    folder_name = os.path.basename(os.path.dirname(img_path))

    # Create the output folder if it does not exist
    output_folder = os.path.join(output_dir, folder_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the input image with the output image name in the output folder
    output_image_path = os.path.join(output_folder, output_image_name)
    Image.open(img_path).save(output_image_path)


def analyze_emotion_deepface2(img_path, input_dir, output_dir):
    # Perform emotion recognition
    result = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)

    # Get the maximum emotion and its score
    max_emotion = sorted(result[0]["emotion"].items(), key=lambda x: x[1], reverse=True)[0]
    max_emotion_name, max_emotion_score = max_emotion

    # Create the output image name
    image_name = os.path.splitext(os.path.basename(img_path))[0]
    output_image_name = f"{image_name}_{max_emotion_name}_{max_emotion_score:.2f}.jpg"

    # Create the output folder if it does not exist
    folder_name = os.path.basename(os.path.dirname(img_path))
    output_folder = os.path.join(output_dir, folder_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the input image with the output image name in the output folder
    output_image_path = os.path.join(output_folder, output_image_name)
    Image.open(img_path).save(output_image_path)

    # Save the emotions dictionary to a txt file
    emotions_path = os.path.join(output_folder, f"{image_name}_emotions.txt")
    with open(emotions_path, "w") as f:
        json.dump(result[0]["emotion"], f)




# def detect_emotion_fer(image_path, input_dir, output_dir):
#     # Load the input image
#     img = cv2.imread(image_path)

#     # Create an instance of the FER detector
#     detector = FER()

#     # Detect emotions in the input image using FER
#     result = detector.detect_emotions(img)

#     # Get the emotion with the highest probability
#     emotion = max(result[0]['emotions'], key=result[0]['emotions'].get)

#     # Get the percentage of the detected emotion
#     percentage = result[0]['emotions'][emotion] * 100

#     image_name = os.path.splitext(os.path.basename(img_path))[0]
#     output_image_name = f"{image_name}_{emotion}_{percentage:.2f}.jpg"

#     # Get the folder name of the input image
#     folder_name = os.path.basename(os.path.dirname(img_path))

#     # Create the output folder if it does not exist
#     output_folder = os.path.join(output_dir, folder_name)
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Save the input image with the output image name in the output folder
#     output_image_path = os.path.join(output_folder, output_image_name)
#     Image.open(img_path).save(output_image_path)

def remove_lines(path):
# Set the path to the directory containing the images and text files
    directory_path = path

    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Read the contents of the text file
            with open(os.path.join(directory_path, filename), "r") as f:
                lines = f.readlines()

            # Remove any empty lines
            lines = [line.strip() for line in lines if line.strip() != ""]

            # Write the modified contents back to the text file
            with open(os.path.join(directory_path, filename), "w") as f:
                f.write("\n".join(lines))


In [ ]:
def main1():
    # Specify the input directory containing images
    input_dir = "/content/drive/MyDrive/Final Code/uploads"

    # Specify the output directory for the coordinates and output images
    output_dir = "/content/drive/MyDrive/Final Code/uploads_detected"

    # Create the output directory if it doesn't already exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get a list of all image paths in the input directory
    image_paths = [os.path.join(input_dir, filename) for filename in os.listdir(input_dir) if filename.endswith(".jpg")]

    # Process each image
    for image_path in image_paths:
        detect_faces(image_path, output_dir)


def main2():

    input_dir = "/content/drive/MyDrive/Final Code/uploads_detected"
    output_dir = "/content/drive/MyDrive/Final Code/uploads_extracted"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all files in the input directory
    for filename in os.listdir(input_dir):
        # Check if the file is a JPG image
#         print(f"Processing file: {filename}")
        if filename.endswith('.jpg'):
            # Extract the path of the corresponding TXT file
            txt_filename = os.path.splitext(filename)[0] + '_coordinates.txt'
            txt_path = os.path.join(input_dir, txt_filename)
            print(f"Text file: {txt_path}")


            # Check if the TXT file exists
            if os.path.isfile(txt_path):
#                 print("HELLO")
                # Call the extract_faces() function with the image and TXT paths, and the output directory
                image_path = os.path.join(input_dir, filename)
#                 print(f"Processing image: {filename}, txt file: {txt_filename}")
                extract_faces(image_path, txt_path, output_dir)

# def main3():

#     input_dir = 'faces_extracted'
#     output_dir = 'deepface_emotions'

#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     # Loop through all folders in the input directory
#     for folder_name in os.listdir(input_dir):
#         input_folder_path = os.path.join(input_dir, folder_name)

#         # Check if the path is a folder and not a file
#         if os.path.isdir(input_folder_path):
#             print(f"Processing folder: {folder_name}")

#             # Create the corresponding output folder if it does not exist
#             output_folder_path = os.path.join(output_dir, folder_name)
#             if not os.path.exists(output_folder_path):
#                 os.makedirs(output_folder_path)

#             # Loop through all files in the input folder
#             for filename in os.listdir(input_folder_path):
#                 # Check if the file is a JPG image
#                 if filename.endswith('.jpg'):
#                     print(f"Processing image: {filename}")
#                     # Analyze the emotion of the image and save it in the corresponding output folder
#                     input_image_path = os.path.join(input_folder_path, filename)
#                     detect_emotion_fer(input_image_path, input_folder_path, output_dir)

def main3():

    input_dir = '/content/drive/MyDrive/Final Code/uploads_extracted'
    output_dir = '/content/drive/MyDrive/Final Code/uploads_emotions'

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all folders in the input directory
    for folder_name in os.listdir(input_dir):
        input_folder_path = os.path.join(input_dir, folder_name)

        # Check if the path is a folder and not a file
        if os.path.isdir(input_folder_path):
            print(f"Processing folder: {folder_name}")

            # Create the corresponding output folder if it does not exist
            output_folder_path = os.path.join(output_dir, folder_name)
            if not os.path.exists(output_folder_path):
                os.makedirs(output_folder_path)

            # Loop through all files in the input folder
            for filename in os.listdir(input_folder_path):
                # Check if the file is a JPG image
                if filename.endswith('.jpg'):
                    print(f"Processing image: {filename}")
                    # Analyze the emotion of the image and save it in the corresponding output folder
                    input_image_path = os.path.join(input_folder_path, filename)
                    analyze_emotion_deepface2(input_image_path, input_folder_path, output_dir) #calling deepface2

def main4():
    path = "/content/drive/MyDrive/Final Code/uploads_detected"
    remove_lines(path)


if __name__ == "__main__":
    #main1()
    #main4()
    #main2()
    main3()


Processing folder: 172_output
Processing image: 172_output_face_33.jpg


Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


Processing image: 172_output_face_2.jpg


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 14.32it/s]


Processing image: 172_output_face_11.jpg


...

Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  7.71it/s]


Processing image: 54_output_face_53 (1).jpg


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 11.01it/s]


Processing image: 54_output_face_74 (1).jpg


Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  9.06it/s]


In [ ]:
import os
import json

directory = "/content/drive/MyDrive/Final Code/uploads_detected"

def calculate_average_emotions(folder_path):
    emotions_sum = {}
    num_files = 0

    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r") as file:
                data = json.load(file)

            # Add the emotion values to the sum
            for emotion, value in data.items():
                emotions_sum[emotion] = emotions_sum.get(emotion, 0) + value

            num_files += 1

    # Calculate the average emotion values
    average_emotions = {emotion: value / num_files for emotion, value in emotions_sum.items()}
    return average_emotions

# Iterate over all folders in the directory
for folder_name in os.listdir(directory):
    folder_path = os.path.join(directory, folder_name)
    if os.path.isdir(folder_path):
        average_emotions = calculate_average_emotions(folder_path)

        # Save the average emotions to a text file with the same name as the folder
        output_file_path = os.path.join(folder_path, folder_name + ".txt")
        with open(output_file_path, "w") as output_file:
            print(output_file_path)
            json.dump(average_emotions, output_file)

In [ ]:
import os
import json

directory = "/content/drive/MyDrive/Final Code/uploads_emotions"

# Iterate over all folders in the directory
for folder_name in os.listdir(directory):
    folder_path = os.path.join(directory, folder_name)
    if os.path.isdir(folder_path):
        average_emotions = calculate_average_emotions(folder_path)

        # Find the top two emotions with the highest values
        sorted_emotions = sorted(average_emotions.items(), key=lambda x: x[1], reverse=True)
        top_emotion, top_value = sorted_emotions[0]
        second_emotion, second_value = sorted_emotions[1]

        # Check the absolute difference between the top two values
        if abs(top_value - second_value) < 5:
            predicted_emotions = {top_emotion: top_value, second_emotion: second_value}
        else:
            predicted_emotions = {top_emotion: top_value}

        # Save the predicted emotions to a text file with the naming format <folder_name>_predictions.txt
        output_file_path = os.path.join(folder_path, f"{folder_name}_predictions.txt")
        with open(output_file_path, "w") as output_file:
            json.dump(predicted_emotions, output_file)

        # Print the predicted emotions and their values
        print(f"Predicted emotions for {folder_name}:")
        for emotion, value in predicted_emotions.items():
            print(f"{emotion}: {value}")
        print()

Predicted emotions for 172_output:
sad: 34.83005497132863

Predicted emotions for 150_output:
fear: 49.87854189853535

Predicted emotions for 244_output:
sad: 52.055166686995975

Predicted emotions for 152_output:
happy: 37.42937297145331
neutral: 37.411214088936504

Predicted emotions for 219_output:
happy: 35.02923654503406
angry: 31.47241323958043

Predicted emotions for 129_output:
sad: 64.84961663332919

Predicted emotions for 123_output:
fear: 42.10126041143667

Predicted emotions for 161_output:
sad: 35.783421550052914

Predicted emotions for 42_output:
happy: 42.64009339727867

Predicted emotions for 120_output:
sad: 44.21706684056632

Predicted emotions for 191_output:
sad: 39.94096799185728

Predicted emotions for 226_output:
happy: 85.5235477288564

Predicted emotions for 171_output:
happy: 46.940602695906016

Predicted emotions for 90_output:
neutral: 34.18810446503593

Predicted emotions for 78_output:
angry: 46.03216210918311

Predicted emotions for 180_output:
sad: 29.21